In [8]:
# --- Helper Functions for SHA-256 ---

def right_rotate(n, b):
    """Performs a right-rotate (circular shift) on a 32-bit integer."""
    return ((n >> b) | (n << (32 - b))) & 0xffffffff

# The six logical functions as defined in the FIPS 180-4 standard
def choice(x, y, z):
    """The Ch(x, y, z) function."""
    return (x & y) ^ (~x & z)

def majority(x, y, z):
    """The Maj(x, y, z) function."""
    return (x & y) ^ (x & z) ^ (y & z)

def sigma0(x):
    """The Σ0(x) function (lowercase sigma)."""
    return right_rotate(x, 2) ^ right_rotate(x, 13) ^ right_rotate(x, 22)

def sigma1(x):
    """The Σ1(x) function (lowercase sigma)."""
    return right_rotate(x, 6) ^ right_rotate(x, 11) ^ right_rotate(x, 25)

def gamma0(x):
    """The σ0(x) function (uppercase sigma, often called 'gamma' in code)."""
    return right_rotate(x, 7) ^ right_rotate(x, 18) ^ (x >> 3)

def gamma1(x):
    """The σ1(x) function (uppercase sigma, often called 'gamma' in code)."""
    return right_rotate(x, 17) ^ right_rotate(x, 19) ^ (x >> 10)
# --- Constants for SHA-256 ---

# h_constants: Initial hash values
# (First 32 bits of the fractional parts of the square roots of the first 8 primes)
h_constants = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

# k_constants: Round constants
# (First 32 bits of the fractional parts of the cube roots of the first 64 primes)
k_constants = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

In [9]:
def sha256_from_scratch_manual(message_bytes):
    """Computes the SHA-256 hash without using the struct library."""
    
    # --- 1. Pre-processing (Padding) ---
    
    original_len_bits = len(message_bytes) * 8
    padded_message = message_bytes + b'\x80'
    padding_len = (56 - len(padded_message) % 64) % 64
    padded_message += b'\x00' * padding_len
    
    # Manually convert the length to 8 big-endian bytes
    length_bytes = bytearray(8)
    for i in range(8):
        length_bytes[i] = (original_len_bits >> (56 - i * 8)) & 0xff
    padded_message += length_bytes
    # -----------------------------------------------------------------

    # --- 2. Process the message in 512-bit (64-byte) chunks ---
    
    h = list(h_constants)
    
    for i in range(0, len(padded_message), 64):
        chunk = padded_message[i:i+64]
        
        # Manually parse the 64-byte chunk into 16 32-bit integers
        w = [0] * 64
        for j in range(16):
            offset = j * 4
            w[j] = ((chunk[offset] << 24) |
                    (chunk[offset+1] << 16) |
                    (chunk[offset+2] << 8) |
                    (chunk[offset+3]))
        # -----------------------------------------------------------------
            
        # Extend the 16 words into 64 words for the message schedule
        for t in range(16, 64):
            w[t] = (gamma1(w[t-2]) + w[t-7] + gamma0(w[t-15]) + w[t-16]) & 0xffffffff
            
        # Initialize working variables
        a, b, c, d, e, f, g, H = h
        
        # --- 3. Compression Loop ---
        for t in range(64):
            t1 = (H + sigma1(e) + choice(e, f, g) + k_constants[t] + w[t]) & 0xffffffff
            t2 = (sigma0(a) + majority(a, b, c)) & 0xffffffff
            
            H = g; g = f; f = e;
            e = (d + t1) & 0xffffffff
            d = c; c = b; b = a;
            a = (t1 + t2) & 0xffffffff
            
        # Update hash values
        h[0] = (h[0] + a) & 0xffffffff
        h[1] = (h[1] + b) & 0xffffffff
        h[2] = (h[2] + c) & 0xffffffff
        h[3] = (h[3] + d) & 0xffffffff
        h[4] = (h[4] + e) & 0xffffffff
        h[5] = (h[5] + f) & 0xffffffff
        h[6] = (h[6] + g) & 0xffffffff
        h[7] = (h[7] + H) & 0xffffffff
        
    # --- 4. Produce final hash value ---
    return ''.join(f'{val:08x}' for val in h)

print("Manual SHA-256 function is defined.")

Manual SHA-256 function is defined.


In [10]:
import hashlib

# --- Test Case 1: An empty string ---
test_string_1 = ""
my_hash_1 = sha256_from_scratch_manual(test_string_1.encode('utf-8'))
hashlib_hash_1 = hashlib.sha256(test_string_1.encode('utf-8')).hexdigest()

print(f"Test String 1: '' (empty string)")
print(f"My Manual Implementation: {my_hash_1}")
print(f"hashlib Result:         {hashlib_hash_1}")
print(f"Match: {my_hash_1 == hashlib_hash_1}\n")

# --- Test Case 2: A simple string ---
test_string_2 = "Hello World"
my_hash_2 = sha256_from_scratch_manual(test_string_2.encode('utf-8'))
hashlib_hash_2 = hashlib.sha256(test_string_2.encode('utf-8')).hexdigest()

print(f"Test String 2: '{test_string_2}'")
print(f"My Manual Implementation: {my_hash_2}")
print(f"hashlib Result:         {hashlib_hash_2}")
print(f"Match: {my_hash_2 == hashlib_hash_2}\n")

Test String 1: '' (empty string)
My Manual Implementation: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
hashlib Result:         e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Match: True

Test String 2: 'Hello World'
My Manual Implementation: a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e
hashlib Result:         a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e
Match: True



In [11]:
import math

# --- 1. Define initial constants and tables ---

# The S-table specifies the per-round shift amounts
S = [
    7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22, 7, 12, 17, 22,
    5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20, 5,  9, 14, 20,
    4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23, 4, 11, 16, 23,
    6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21, 6, 10, 15, 21
]

# The T-table is computed from the sine function
T = [int(abs(math.sin(i + 1)) * 2**32) & 0xFFFFFFFF for i in range(64)]


# --- 2. Define initial hash values (the chaining variables A, B, C, D) ---
A = 0x67452301
B = 0xefcdab89
C = 0x98badcfe
D = 0x10325476


# --- 3. Define helper functions ---

def left_rotate(x, n):
    """Performs a left-rotate (circular shift) on a 32-bit integer."""
    x &= 0xFFFFFFFF  # Ensure x is 32-bit
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

print("MD5 constants and helper functions are defined.")

MD5 constants and helper functions are defined.


In [12]:
def md5_from_scratch(message_bytes):
    """Computes the MD5 hash of a given byte string."""
    
    # --- 1. Pre-processing (Padding) ---
    original_len_bits = (len(message_bytes) * 8) & 0xffffffffffffffff
    padded_message = message_bytes + b'\x80'
    
    padding_len = (56 - len(padded_message) % 64) % 64
    padded_message += b'\x00' * padding_len
    
    # Append original length as a 64-bit little-endian integer
    length_bytes = bytearray(8)
    for i in range(8):
        length_bytes[i] = (original_len_bits >> (i * 8)) & 0xff
    padded_message += length_bytes

    # --- 2. Process the message in 512-bit (64-byte) chunks ---
    h0, h1, h2, h3 = A, B, C, D # Initialize chaining variables

    for i in range(0, len(padded_message), 64):
        chunk = padded_message[i:i+64]
        
        # Initialize hash values for this chunk
        a, b, c, d = h0, h1, h2, h3

        # Parse chunk into 16 32-bit words (little-endian)
        M = []
        for j in range(0, 64, 4):
            word = (chunk[j] | (chunk[j+1] << 8) | (chunk[j+2] << 16) | (chunk[j+3] << 24))
            M.append(word)

        # --- 3. Main Compression Loop (64 rounds) ---
        for j in range(64):
            if 0 <= j <= 15:
                f = (b & c) | (~b & d)
                g = j
            elif 16 <= j <= 31:
                f = (d & b) | (~d & c)
                g = (5 * j + 1) % 16
            elif 32 <= j <= 47:
                f = b ^ c ^ d
                g = (3 * j + 5) % 16
            elif 48 <= j <= 63:
                f = c ^ (b | ~d)
                g = (7 * j) % 16
            
            f = (f + a + T[j] + M[g]) & 0xFFFFFFFF
            a = d
            d = c
            c = b
            b = (b + left_rotate(f, S[j])) & 0xFFFFFFFF

        # Update chaining variables
        h0 = (h0 + a) & 0xFFFFFFFF
        h1 = (h1 + b) & 0xFFFFFFFF
        h2 = (h2 + c) & 0xFFFFFFFF
        h3 = (h3 + d) & 0xFFFFFFFF

    # --- 4. Produce final hash value (in little-endian) ---
    def to_little_endian_hex(val):
        """Converts a 32-bit integer to a little-endian hex string."""
        return ''.join(f'{(val >> (i * 8)) & 0xff:02x}' for i in range(4))

    return to_little_endian_hex(h0) + to_little_endian_hex(h1) + to_little_endian_hex(h2) + to_little_endian_hex(h3)

print("Main MD5 function is defined.")

Main MD5 function is defined.


In [13]:
import hashlib

# --- Test Case 1: An empty string ---
test_string_1 = ""
my_hash_1 = md5_from_scratch(test_string_1.encode('utf-8'))
hashlib_hash_1 = hashlib.md5(test_string_1.encode('utf-8')).hexdigest()

print(f"Test String 1: '' (empty string)")
print(f"My Implementation: {my_hash_1}")
print(f"hashlib Result:    {hashlib_hash_1}")
print(f"Match: {my_hash_1 == hashlib_hash_1}\n")

# --- Test Case 2:  ---
test_string_2 = "The quick brown fox jumps over the lazy dog"
my_hash_2 = md5_from_scratch(test_string_2.encode('utf-8'))
hashlib_hash_2 = hashlib.md5(test_string_2.encode('utf-8')).hexdigest()

print(f"Test String 2: '{test_string_2}'")
print(f"My Implementation: {my_hash_2}")
print(f"hashlib Result:    {hashlib_hash_2}")
print(f"Match: {my_hash_2 == hashlib_hash_2}\n")

# --- Test Case 3:  ---
test_string_3 = "The five boxing wizards jump quickly."
my_hash_3 = md5_from_scratch(test_string_3.encode('utf-8'))
hashlib_hash_3 = hashlib.md5(test_string_3.encode('utf-8')).hexdigest()

print(f"Test String 3: '{test_string_3}'")
print(f"My Implementation: {my_hash_3}")
print(f"hashlib Result:    {hashlib_hash_3}")
print(f"Match: {my_hash_3 == hashlib_hash_3}")

Test String 1: '' (empty string)
My Implementation: d41d8cd98f00b204e9800998ecf8427e
hashlib Result:    d41d8cd98f00b204e9800998ecf8427e
Match: True

Test String 2: 'The quick brown fox jumps over the lazy dog'
My Implementation: 9e107d9d372bb6826bd81d3542a419d6
hashlib Result:    9e107d9d372bb6826bd81d3542a419d6
Match: True

Test String 3: 'The five boxing wizards jump quickly.'
My Implementation: e11ccd1b8abc3ffcefb9188757995751
hashlib Result:    e11ccd1b8abc3ffcefb9188757995751
Match: True


In [14]:
def bbs_generator(seed, p, q):
    """
    A Blum-Blum-Shub (BBS) generator.
    Yields a sequence of cryptographically secure random bits.
    """
    # 1. Validate that p and q are congruent to 3 (mod 4)
    if p % 4 != 3 or q % 4 != 3:
        raise ValueError("p and q must both be congruent to 3 (mod 4)")
        
    # 2. Calculate the modulus M
    M = p * q
    
    X = seed
    while True:
        # 3. The core formula: X_next = X^2 mod M
        X = pow(X, 2, M) # pow(x, y, z) is an efficient way to compute (x**y) % z
        
        # 4. Yield the least significant bit (LSB)
        yield X % 2

# Choose two small primes that are 3 mod 4 (e.g., 7, 11, 19, 23, 31...)
p = 11
q = 19
seed = 3 # A starting number that isn't a factor of p or q

# Create the generator instance
bbs_gen = bbs_generator(seed, p, q)

print(f"--- Generating 16 random bits using BBS (p={p}, q={q}) ---")
random_bits = []
for _ in range(16):
    random_bits.append(next(bbs_gen))

print(f"Generated bits: {''.join(map(str, random_bits))}")

# We can combine these bits to form a random integer
random_number = 0
for bit in random_bits:
    random_number = (random_number << 1) | bit # Left shift and add the new bit

print(f"\nThe 16 bits combined to form the integer: {random_number}")

--- Generating 16 random bits using BBS (p=11, q=19) ---
Generated bits: 1100000101101100

The 16 bits combined to form the integer: 49516


In [15]:
def lcg_generator(seed, a, c, m):
    
    # Start with the initial seed value
    current_x = seed
    
    while True:
        # Apply the LCG formula: X_n+1 = (a * X_n + c) mod m
        current_x = (a * current_x + c) % m
        yield current_x

# --- Example Usage ---

# Parameters from the popular "Numerical Recipes" LCG
MODULUS    = 2**32
MULTIPLIER = 1664525
INCREMENT  = 1013904223
SEED       = 12345 

# 1. Create the generator instance
random_gen = lcg_generator(seed=SEED, a=MULTIPLIER, c=INCREMENT, m=MODULUS)

# 2. Generate and print the first 10 numbers from the sequence
print(f"Generating 10 pseudorandom numbers with seed={SEED}:\n")
for i in range(10):
    random_number = next(random_gen)
    print(f"Number {i+1}: {random_number}")

Generating 10 pseudorandom numbers with seed=12345:

Number 1: 87628868
Number 2: 71072467
Number 3: 2332836374
Number 4: 2726892157
Number 5: 3908547000
Number 6: 483019191
Number 7: 2129828778
Number 8: 2355140353
Number 9: 2560230508
Number 10: 3364893915
